<a href="https://colab.research.google.com/github/IntenseSunlight/replika_backup_colab/blob/main/Replika_log_backup_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Acknowledgements
- based upon the **replika_backup** repo from @Hotohori and other authors <br>
(https://github.com/Hotohori/replika_backup)
- modifications for Colab notebook: <br> (https://github.com/IntenseSunlight/replika_backup_colab)

In [ ]:
#@title 1. Install/Import requirements { display-mode: "form" }
#@markdown - **websocket**:  used to handle websocket interactions
#@markdown - **ipyfilechooser**: file browser for notebooks
#@markdown - **replika_backup_colab**: Git repository for *ReplickaWebSocketHandler*
#@markdown  
#@markdown **Note**: Run this cell with the **Play Button**: 👉 ![play-button.png](data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAABQAAAAUCAYAAACNiR0NAAAAAXNSR0IArs4c6QAAAARnQU1BAACxjwv8YQUAAAAJcEhZcwAADsMAAA7DAcdvqGQAAAE7SURBVDhPrZQxjoMwEEUHp4UWuAX0HIBDIMEdKKhTI8EdQOIQ3AJuAT397v5ZGzmJMSS7TyKWHfHkPx7jfP1ABuZ55meaJh7DMKQoivYRj4kX4bIsVNc1S2xA3DQNjzoPwr7vqes6OTsHsjRNKc9zuaIJ35XpFEWxSwV+EPNTGRjHcS8RC1GzI55rZELVHQjdbgKFR6QzIIVHrOsql8xgh6jPFSkLYb4CpMMwWEuAnr35vn/ftk0uvaLvzHVdSpKEPM8zlslxnN9DeQfs8CgV1kUQBHJ6Dl4oy5LbxASaXMRxLKd2IMmyzNoR2L04uuQ66DFbryqQliPbpLaIOnAgMt9l1AZx/kLbtizlU0b2qqr4j09Aa6mUe9tg4cpteAbvGD9fCsT/tw+sAlLccYwQKzkOEG2GNCriI0TfQAWz5tFeveoAAAAASUVORK5CYII=) <br>
#@markdown *(some cells will run autmatically when you change an option)*

# not installed by default in Colab notebook
!pip install websocket-client ipyfilechooser 

# obtain the target repo for resources
repo_source = 'https://github.com/IntenseSunlight'
repo_name = 'replika_backup_colab'
commit_hash = 'HEAD'
repo_http = repo_source + '/' + repo_name + '.git'

!git clone $repo_http
!cd $repo_name && git checkout $commit_hash && cd ..

import os, sys
if not os.path.exists(repo_name):
  valid_session = False
  raise(Exception("Failed: could not clone repo to local notebook; cannot continue"))
else:
  sys.path.append(os.path.join(os.getcwd(),repo_name))
  from ReplikaWebSocketHandler import ReplikaWebSocketHandler
  print("\nCompleted step 1!\nSuccessfully imported ReplikaWebSocketHandler")


In [ ]:
#@title 2. Replika session key { run: "auto", display-mode: "form" }
#@markdown # Obtain session settings
#@markdown - Login to the replika website (https://my.replika.com) (browser: Chrome or Firefox)
#@markdown ---
#@markdown (Based on the instructions from [Replika_backup](https://github.com/Hotohori/replika_backup/blob/main/SETUP_GUIDE.md))
#@markdown - Open a Chrome, Edge, Firefox browser and login to your Replika account.
#@markdown - Press Ctrl + Shift + i inside the browser to open the developer tools window.
#@markdown - Follow the red numbers on the picture. First image is Chrome and Edge, second is Firefox
#@markdown - Copy the this **init** section key from this *inspection* browser
#@markdown - Note: it will look like this example:
#@markdown   -  {"event name": "init", "payload":{"device_id": "123456789",...,"user_id":"123456789", "auth_token":"123456789", "security_token":"123456789"..}}
#@markdown --- 
#@markdown Example to obtain **init** key, Chrome/Edge/Firefox: <br>
#@markdown ![chrome-edge-firefox.png](...)

#@markdown ---
#@markdown Paste session key here:
init_session_key = "" #@param {type:"string"}
#@markdown ...after entering your key, this should start automatically <br>
#@markdown *(if not, press the  **Play** button at the top of this cell)*

init = None

if not init_session_key:
  print("Failed: the key was blank")
else:
  try:
    import json
    _ = json.loads(init_session_key)  # only to check if it parses
    init = init_session_key
    print('Completed step 2!\nInit key parse Ok')
  except:
    raise(Exception("Something failed: could not parse init string (check that it was fully copied)"))
    print("Correct issues before continuing")

In [ ]:
#@title 3. (optional) Obtain previous log { run: "auto", display-mode: "form" }
#@markdown This step enables you to upload a previous Replika log file. 
#@markdown If a previous log file does not exist, this step can be skipped
#@markdown - Options:
#@markdown   - Upload log file from local computer <br>
#@markdown   or
#@markdown   - Connect to Gdrive to fetch previous log file
#@markdown ---
options = "Upload from local" #@param ["Skip step", "Upload from local", "Retrieve from Gdrive"]

import os
import ipywidgets as widgets
from shutil import copy

prev_log_file_path = None
if options == "Upload from local":
  from google.colab import files
  res = files.upload()
  if res:
    prev_log_file_path = os.path.join(os.getcwd(),list(res.keys())[0])
    print("Selected file was: ", prev_log_file_path)
  else:
    prev_log_file_path = None
    print("Cancelled operation")
  
elif options == "Retrieve from Gdrive":
  # attempt to mount Gdrive (this requires user permissions)
  from ipyfilechooser import FileChooser
  from google.colab import drive
  basefolder = '/content/gdrive'
  drive.mount(basefolder)

  def file_complete(*args, **kwargs):
    global prev_log_file_path
    prev_log_file_path = fc.selected
    print(f"\nCompleted step 3!\nPrevious logfile located: {prev_log_file_path}")

  fc = FileChooser(os.path.join(basefolder,'MyDrive'))
  fc.filter_pattern = "*.csv"
  fc.register_callback(file_complete)

  hbox = widgets.HBox([
      widgets.Label("Select file from Gdrive -->"),
      fc
  ])
  display(hbox)
else:
  print("\nCompleted step 3!\nSkipping step")



In [ ]:
#@title 4. Extract messages { run: "auto", display-mode: "form" }
#@markdown This step extracts the latest messages from my.replika.com <br>
#@markdown Select one of the following possibilites:
#@markdown - **A**) Extract new; add to prior log
#@markdown   - If a previous log exists (from step 3), add messages to the top of the log file <br>
#@markdown   - \* *(Entries in log file are stored in reverse chronological order, i.e. newest entries at top of the file)*
#@markdown - **B**) Extract messages up to specified date
#@markdown - **C**) Extract a limited number of messages
#@markdown - **D**) Extract all possible messages (to earliest possible history)
extraction_option = "A) Extract new; add to prior log" #@param ["A) Extract new; add to prior log", "B) Extract to specific date", "C) Extract specific number of messages", "D) Extract all possible messages"]
#@markdown ---
#@markdown ### **Options** *(for steps **B**, **C**, **D**)*
#@markdown Default file name (*for steps* **B**, **C**, and **D** *only*)
default_file_name = "chat_log" #@param {type:"string"}
#@markdown Extract from *now* until *limitdate*: (*for step* **B** *only*)
input_limitdate = "2022-12-01" #@param {type:"date"}
#@markdown Extract limited number of messages (*for step* **C** *only*)
input_limitmsgs = 50 #@param {type:"integer"}


# -------------------------------------------------------------
# helper functions
# -------------------------------------------------------------
import os
from shutil import copy
from dateutil.parser import parse as date_parse
from datetime import datetime
from threading import Event

curr_def = dir()

def run_extraction(
    init,
    file_name,
    limitdate=None,
    limitmsgs=None,
    lastmsgid=None
):
  import csv
  import websocket

  # Note: when this runs, sometimes it creates strange errors which are 
  # reported to the STDOUT and not caught by the error handler
  # usually they are non-fatal and can be ignored
  # (if you know how to get rid of them, feedback is welcome :)
  with open(file_name, 'w', newline='', encoding='UTF-8') as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(['Timestamp', 'From', 'Text', 'Reaction', 'ID'])

    ws_handler = ReplikaWebSocketHandler(
      init=init,
      limitdate=limitdate,
      limitmsgs=limitmsgs,
      lastmsgid=lastmsgid,
      writer=writer.writerow,
      logger=print
    )

    print('Open websocket to your Replika AI.', end='')
    websocket.enableTrace(False)
    ws = websocket.WebSocketApp(
      "wss://ws.replika.ai/v17",
      on_open=ws_handler.on_open,
      on_message=ws_handler.on_message,
      on_error=ws_handler.on_error,
      on_close=ws_handler.on_close
    )
    ws.run_forever()
  return count_number_lines(file_name)

def check_auth():
  return ('init' in curr_def) and (init is not None)  

def rename_to_old(
    file_name
):
  old_file_name = file_name + '.old'
  os.rename(file_name, old_file_name )
  return old_file_name

def get_last_message_time_id(
    file_name
):
  with open(file_name,'r',encoding='UTF-8') as f:
    res = f.readline().strip().split(',')[0]  # first line is header
    assert res == 'Timestamp', \
      f"Failed to read {file_name}; did not contain an appropriate header"
    res = f.readline().strip().split(',')
    t = res[0].split('T')[0]  # should contain last message timestamp date
    id = res[-1]  # should be last message id

  return t, id

def append_old_to_new(
    new_file,  # final file name, contains new messages
    old_file,  # a temporarily renamed file of the old messages appended at end
    remove_old=True
):
  skip = 1  # first line of old file is header
  with open(new_file,'a',encoding='UTF-8') as nf:
    with open(old_file,'r',encoding='UTF-8') as of:
      for l in of:
        if skip:
          skip -= 1  
          continue
        nf.write(l)

  if remove_old:
    os.unlink(old_file)

def ok_to_write_file(file_name):
  if os.path.exists(file_name):
    print(f"A log file with name='{file_name}' already exists")
    res = input("Remove old file? Please type 'y' to continue: ")
    if res.lower() == 'y':
      os.unlink(file_name)
      if os.path.exists(file_name):
        print(f"Failed to remove '{file_name}'")
        return False
      else:
        print("Removed file..continuing")
        return True
    else:
      print("Choose a new name or remove the previous file to continue")
      return False
  else:
    return True

def count_number_lines(file_name):
  with open(file_name, 'r', encoding='UTF-8') as f:
    return sum(1 for _ in f)

# -------------------------------------------------------------

if not default_file_name.endswith('.csv'):
  default_file_name = default_file_name + '.csv'

file_name = None        # prevents step 5 from happening if no file was defined

# process: user selection, check authorization
# check if authorized
if not check_auth():
  print("Cannot continue: user credentials have not been defined; Re-run step 2")

# Option A: append to old log
elif extraction_option.startswith("A)"):
  # determine if previous step were run
  if ('prev_log_file_path' in dir()) and (prev_log_file_path is not None):
    prev_log_file_path = os.path.abspath(prev_log_file_path)
    file_name = os.path.basename(prev_log_file_path)
    
    # copy file to working directory if not already there
    if os.path.dirname(prev_log_file_path) != os.getcwd():
      copy(prev_log_file_path,os.getcwd())
    
    limitdate, lastmsgid = get_last_message_time_id(file_name)   # get last date, last msg id from file
    old_file_name = rename_to_old(file_name)       # rename current file to .old
    n_lines = run_extraction(                      # perform chat extraction
        init=init,
        file_name=file_name,
        #limitdate=limitdate,
        lastmsgid=lastmsgid
    )
    append_old_to_new(file_name, old_file_name)    # append old log messages to new messages
    if n_lines > 1:
      print("Option A)\nCompleted log extraction:")  # happy ending
      print(f"- Extacted new messages until date {limitdate}")
      print(f"- Appened old chat messages to new log")
      print("\nCompleted step 4!")
    else:
      print("Option A)\nStrange completion:")       # weird ending
      print(f"- it seems no new lines were added to the log file")
      print(f"- check inputs to see if something was invalid")

  else:
    file_name = None   
    print("Cannot use Option A); No previous log file specified (or step 3 not completed)")

# Option B: extract to certain date 
elif extraction_option.startswith("B)"):
  file_name = default_file_name

  if ok_to_write_file(file_name):
    now = datetime.today()
    then = date_parse(input_limitdate)
    diff = now - then

    if diff.days > 0:
      limitdate = input_limitdate
      n_lines = run_extraction(       # perform chat extraction
        init=init,
        file_name=file_name,
        limitdate=limitdate
      )
      if n_lines > 1:
        print("Option B)\nCompleted log extraction:")     # happy ending
        print(f"- Extacted new messages until date {limitdate}")
        print(f"- Saved messages to file: {file_name}")
        print("\nCompleted step 4!")
      else:
        print("Option B)\nStrange completion:")       # weird ending
        print(f"- it seems no new lines were added to the log file")
        print(f"- check inputs to see if something was invalid (possible invalid 'init'?)")

    else:
      file_name = None
      print("Option B)\nIncorrect date supplied; date must be in the past")

# Option C: Process limited number of messages
elif extraction_option.startswith("C)"):
  file_name = default_file_name
  if input_limitmsgs < 1:
    print(f"Cannot process with input_limitmsgs={input_limitmsgs}; value must > 0")
  elif ok_to_write_file(file_name):
    n_lines = run_extraction(     # perform chat extraction
      init=init,
      file_name=file_name,
      limitmsgs=input_limitmsgs
    )
    if n_lines > 1:
      print("Option C)\nCompleted log extraction:")   # happy ending
      print(f"- Extacted {input_limitmsgs} messages")
      print(f"- Saved messages to file: {file_name}")
      print("\nCompleted step 4!")
    else:
      print("Option C)\nStrange completion:")       # weird ending
      print(f"- it seems no new lines were added to the log file")
      print(f"- check inputs to see if something was invalid (possible invalid 'init'?, or 0 messages specified)")

# Option D: Extract all messages to new file
elif extraction_option.startswith("D)"):
  file_name = default_file_name
  if ok_to_write_file(file_name):
    n_lines = run_extraction(     # perform chat extraction
      init=init,
      file_name=file_name
    )
    if n_lines > 1:
      print("Option D)\nCompleted log extraction:")   # happy ending
      print(f"- Extacted all messages")
      print(f"- Saved messages to file: {file_name}")
      print("\nCompleted step 4!")
    else:
      print("Option D)\nStrange completion:")       # weird ending
      print(f"- it seems no new lines were added to the log file")
      print(f"- check inputs to see if something was invalid (possible invalid 'init'?)") 

# Weird ending:
else:
  file_name = None
  print("Option ?)\nStrange! Shouldn't have reached here...Try again")


In [ ]:
#@title 5. Export or save Save to Gdrive { display-mode: "form" }
#@markdown Chat log file can now be exported or saved to Gdrive
#@markdown Opitions:
#@markdown - **A**) Export:
#@markdown   - File downloaded to your local browser
#@markdown - **B**) Save to Gdrive:
#@markdown   - Connection to Gdrive will be made (must be acknowledged in seperate window)
#@markdown   - Select a folder in which to save the chat log (file name already specified)
#@markdown   - If file already exists, you will be asked if it's ok to overwrite it
save_export_option = "A) Export file" #@param ["A) Export file", "B) Save file to Gdrive"]

# -------------------------------------------------------
# Helper class
import ipywidgets as widgets
from typing import Callable

class ConfirmationButton:
  def __init__(
      self,
      on_confirm_callback: Callable = lambda *args:...,
      on_cancel_callback: Callable =lambda *args: ...
  ):
    self._on_confirm_callback = on_confirm_callback
    self._on_cancel_callback = on_cancel_callback

    self._confirm_btn = widgets.Button(description='confirm', icon='check', 
                                        button_style='success', layout=dict(width='auto'))
    self._cancel_btn = widgets.Button(description='cancel', icon='times', 
                                      button_style='warning', layout=dict(width='auto'))
    self._cancel_btn.on_click(lambda *args: self._on_cancel_callback())
    self._confirm_btn.on_click(lambda *args: self._on_confirm_callback())
    self._hbox = widgets.HBox([self._confirm_btn, self._cancel_btn])

  def render(self):
    display(self._hbox)

  def hide(self):
    self._hbox.layout.display = "none"

# -------------------------------------------------------

# Previous step was not completed
if ('file_name' not in dir()) or (file_name is None):
  print("Could not export or save the file: complete step 4 first")

# Does file exist? Maybe it was already moved
elif not os.path.exists(file_name):
  print(f"The file '{file_name}' does not exist. Maybe it was already moved")

# Export file to browser
elif save_export_option.startswith("A)"):
  from google.colab import files
  files.download(file_name)
  print(f"\nOptionB):\n - File '{file_name}' was exported")
  print("\nCompleted step 5!")

# Save to Gdrive
elif save_export_option.startswith("B)"):
  # attempt to mount Gdrive (this requires user permissions)
  from ipyfilechooser import FileChooser
  from google.colab import drive
  from shutil import move

  basefolder = '/content/gdrive'
  drive.mount(basefolder)

  save_folder = None
  save_file_path = None
  confirm = None

  def move_file():
    global file_name, save_folder, save_file_path
    if os.path.exists(save_file_path):
      os.unlink(save_file_path)
      if os.path.exists(save_file_path):  # what?? still here?
        print(f"Failed to move '{file_name}'")
        return False

    move(file_name, save_folder)
    if os.path.exists(save_file_path):  # Ok, it was moved
      print(f"\nCompleted step 5!\nSuccessfully save file to {save_file_path}")
      return True

  def confirm_save():
    move_file()
    confirm.hide()
    print("file was saved")
  
  def cancel_save():
    confirm.hide()
    print("file saving canceled")

  def select_complete(*args, **kwargs):
    global fc, save_folder, save_file_path, confirm
    save_folder = fc.selected_path
    save_file_path = os.path.join(save_folder,file_name)
    print("File will be saved to: ", str(save_file_path))
    if os.path.exists(save_file_path):
      print(f"A log file with name='{file_name}' already exists")
      print("Remove old file?")
      confirm = ConfirmationButton(on_confirm_callback=confirm_save, on_cancel_callback=cancel_save)
      confirm.render()
    else:
      move_file()
 
  fc = FileChooser(os.path.join(basefolder,'MyDrive'))
  fc.show_only_dirs = True
  fc.title = '<b>Select save-to folder</b>'
  fc.register_callback(select_complete)

  hbox = widgets.HBox([
      widgets.Label("Select Gdrive folder -->"),
      fc
  ])
  display(hbox)

else:
  print("Option ?)\nStrange! Shouldn't have reached here...Try again")